## 데이터 출처

[Naver sentiment movie corpus]: https://github.com/e9t/nsmc/

- RNN 모델의 학습을 위해 [Naver sentiment movie corpus] 데이터셋 중 10,000건을 추출하여 사용하였습니다.

In [1]:
# torchtext.legacy를 사용할 수 있는 torchtext 버전 설치
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 5.0 MB/s 
     |████████████████████████████████| 831.4 MB 2.4 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [2]:
#colab 을 이용한 실행시
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# torchtext.legacy : text의 preprocessing 파이프라인 정의
# 1) 토크나이징(Tokenization)
# 2) 단어장 생성(Build Vocabulary)
# 3) 토큰의 수치화(Numericalize all tokens)
# 4) 데이터 로더 생성(Create Data Loader)
from torchtext.legacy import data
import torchtext.datasets as datasets

import pickle
print (torch.__version__)

1.9.0+cu102


#LSTM
Applies a multi-layer long short-term memory (LSTM) RNN to an input sequence.

nn.LSTM(input_size, hidden_size, num_layers, bidirectional, batch_first):
* input_size – The number of expected features in the input x
* hidden_size – The number of features in the hidden state h
* num_layers – Number of recurrent layers. 
* bidirectional – If True, becomes a bidirectional LSTM. Default: False
* batch_first – If True, then the input and output tensors are provided as (batch, seq, feature) instead of (seq, batch, feature). Default: False. Note that this does not apply to hidden or cell states.

In [4]:
class RNN_Text(nn.Module):    
    def __init__(self, embed_num, class_num):
        # super()로 Base Class의 __init__() 호출 (nn.Module 클래스 생성자 호출)
        # super(파생클래스, self).__init__() 파이썬 2.x 문법
        # super().__init__() 파이썬 3.x 문법 둘다 사용 가능
        super(RNN_Text, self).__init__()
        # super().__init__()  
        
        V = embed_num   # 단어 사전의 크기
        C = class_num   # 분류하고자 하는 클래스 개수        
        H = 256         # 히든 사이즈
        D = 100         # 단어벡터 차원 100        
        self.embed = nn.Embedding(V, D)        
        
        # LSTM Layer, bidirectional이므로 출력되는 벡터의 크기는 H * 2
        self.rnn = nn.LSTM(D, H, bidirectional = True)
                 
        # Linear Layer : (512, 2)
        self.out = nn.Linear(H*2, C)
        
    def forward(self, x):
        x = self.embed(x)     # (N, W, D) 문장 x의 단어 벡터값 가져옴
      
        # LSTM 모듈 실행
        # LSTM 입력데이터
        # input x : torch.Size([30, 100, 100]) [시퀀스 길이, 배치 사이즈, Dimension]
        x,(_,__) = self.rnn( x, ( self.h, self.c ) )  # lstm을 거쳐나온 값을 x에 담기

        # output x : torch.Size([30, 100, 512]) [시퀀스 길이, 배치 사이즈, 256 * 2]

        # 최종 Hidden Layer로 Linear 모듈 실행   
        logit = self.out(x[-1])  # 가장 마지막 t 단계의 sequence차원에 있는 마지막 단어 hidden states들이 linear단계로 전달

        # 최종 예측 벡터 크기: [배치 사이즈, C], C: 클래스 개수
        return logit       # logit : torch.Size([100, 2])

    def inithidden(self, b):
        #self.h = Variable(torch.randn(2, b, 256))
        #self.c = Variable(torch.randn(2, b, 256))    
        self.h = torch.randn(2, b, 256)   # [2, batch_size, 256]
        self.c = torch.randn(2, b, 256)   # [2, batch_size, 256]


In [5]:
# train, test dataset을 만들어준다
class mydataset(data.Dataset):
    @staticmethod
    def sort_key(ex):
        return len(ex.text)
    def __init__(self, text_field, label_field, path=None, examples=None, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        if examples is None:
            path = self.dirname if path is None else path
            examples = []
            for i,line in enumerate(open(path,'r',encoding='utf-8')):
                if i==0:      # 첫번째 라인은 skip
                    continue
                line = line.strip().split('\t') # text, label 필드가 /tab으로 구분되어 있다                  
                txt = line[1].split(' ')  # 공백을 기준으로 문자열을 나누어 토큰 리스트를 만든다. line[0]에는 ID
               
                # examples: 학습 텍스트, 라벨 텍스트
                # data.Example : Defines a single training or test example.
                examples += [ data.Example.fromlist( [txt, line[2]],fields ) ]
        # Create a dataset from a list of Examples and Fields.
        # fields : field name, field 
        super(mydataset, self).__init__(examples, fields, **kwargs) 


In [6]:
# Field 객체는 다음과 같은 값을 통하여 데이터의 각 필드를 처리하는 방법을 지정
# fix_length: A fixed length that all examples using this field will be padded to, or None for flexible sequence lengths. 
# sequential: Whether the datatype represents sequential data. If False, no tokenization is applied. Default: True.
# batch_first: Whether to produce tensors with the batch dimension first. Default: False.
##text_field = data.Field(fix_length=20)
text_field = data.Field(fix_length=30)
label_field = data.Field(sequential=False, batch_first = True, unk_token = None)

# 학습데이터 Dataset
train_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/Colab Notebooks/aivle/data/nsm/small_ratings_train_tok.txt')
# 테스트데이터 Dataset
test_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/Colab Notebooks/aivle/data/nsm/small_ratings_test_tok.txt')

text_field.build_vocab(train_data)    # Construct the Vocab object 
label_field.build_vocab(train_data)   # Construct the Vocab object 

# Create Iterator objects for train data, test data
train_iter, test_iter = data.Iterator.splits(
                            (train_data, test_data), 
                            batch_sizes=(100, 1), repeat=False)#, device = -1)
len(text_field.vocab)

21893

In [7]:
rnn = RNN_Text(len(text_field.vocab),2)     # embed_num, class_num
optimizer = torch.optim.Adam(rnn.parameters())
rnn.train()

RNN_Text(
  (embed): Embedding(21893, 100)
  (rnn): LSTM(100, 256, bidirectional=True)
  (out): Linear(in_features=512, out_features=2, bias=True)
)

In [8]:
%%time
bool_debug = True    # 텐서의 차원을 출력할 경우 True로 설정
print_idx = 3        # 출력 횟수
for epoch in range(10):
    
    totalloss = 0
    for batch in train_iter:
        optimizer.zero_grad()
        
        txt = batch.text        # torch.Size([20, 100])
        label = batch.label     # torch.Size([100])
        
        if bool_debug and print_idx > 0:
          print ("txt.shape:", txt.shape)
          print_idx -= 1

        # inithiddend : hidden state, cell state 초기화 함수
        rnn.inithidden(txt.size(1))   # 배치 사이즈를 전달
        # 학습 실행
        pred = rnn(txt)
        
        if bool_debug and print_idx > 0:
          print("pred.shape:", pred.shape)
          print("label.shape:", label.shape)
          print_idx -= 1        

        loss = F.cross_entropy(pred, label)
        totalloss += loss.data
        
        loss.backward()
        optimizer.step()
        
    print(epoch,'epoch')  
    print('loss : {:.3f}'.format(totalloss.numpy()))
       
torch.save(rnn,'/content/gdrive/My Drive/Colab Notebooks/aivle/model/rnn_model.pt')

txt.shape: torch.Size([30, 100])
pred.shape: torch.Size([100, 2])
label.shape: torch.Size([100])
txt.shape: torch.Size([30, 100])
0 epoch
loss : 69.748
1 epoch
loss : 69.157
2 epoch
loss : 63.637
3 epoch
loss : 50.839
4 epoch
loss : 40.270
5 epoch
loss : 32.229
6 epoch
loss : 25.666
7 epoch
loss : 19.634
8 epoch
loss : 15.693
9 epoch
loss : 12.990
CPU times: user 7min 3s, sys: 9.77 s, total: 7min 13s
Wall time: 7min 18s


In [9]:
%%time
bool_debug = True    # 텐서의 차원을 출력할 경우 True로 설정

from sklearn.metrics import classification_report
correct = 0
incorrect = 0
rnn.eval()
y_test = []
prediction = []

# 텐서 차원 확인용
print_tensor_shape = 2
print_idx = 1

for batch in test_iter:
    txt = batch.text            # txt.shape: torch.Size([max_sent_len, 1])
    label = batch.label         # label.shape: torch.Size([1])
    y_test.append(label.data[0])
    
    rnn.inithidden(txt.size(1))
   
    pred = rnn(txt)               # pred.shape: torch.Size([1, 2])
    
    _ , ans = torch.max(pred,dim=1) # ans.shape: torch.Size([1])
    prediction.append(ans.data[0])
    
    
    #---------------------------------------
    # 텐서 형태, 데이터를 출력
    if bool_debug and print_tensor_shape > 0:
      print("-----", print_idx, "-----") 
      print("prediction:", prediction)
      print("y_test:", y_test)
      print("pred.shape:", pred.shape)
      #print("pred.data[0]:", pred.data[0])
      print("pred[0]:", pred[0])
      print("pred[0][0]:", pred[0][0])
      print("pred[0][1]:", pred[0][1])
      print("ans.data[0]:", ans.data[0])
      print("ans.shape:", ans.shape)
      print("txt.shape:", txt.shape)
      print("label.shape:", label.shape)
      print("label.data[0]:", label.data[0])
      
      print()
      print_tensor_shape -= 1
      print_idx += 1
      #---------------------------------------

    if ans.data[0] == label.data[0]:  # ans.data[0]: tensor(0) 또는 tensor(1)
        correct += 1    
    else:
        incorrect += 1
    
print ('correct : ', correct)
print ('incorrect : ', incorrect)
print(classification_report(torch.tensor(y_test), 
                            torch.tensor(prediction), 
                            digits=4, 
                            target_names=['negative', 'positive']))


----- 1 -----
prediction: [tensor(1)]
y_test: [tensor(1)]
pred.shape: torch.Size([1, 2])
pred[0]: tensor([-0.4761,  0.4260], grad_fn=<SelectBackward>)
pred[0][0]: tensor(-0.4761, grad_fn=<SelectBackward>)
pred[0][1]: tensor(0.4260, grad_fn=<SelectBackward>)
ans.data[0]: tensor(1)
ans.shape: torch.Size([1])
txt.shape: torch.Size([30, 1])
label.shape: torch.Size([1])
label.data[0]: tensor(1)

----- 2 -----
prediction: [tensor(1), tensor(1)]
y_test: [tensor(1), tensor(1)]
pred.shape: torch.Size([1, 2])
pred[0]: tensor([-1.0454,  1.1015], grad_fn=<SelectBackward>)
pred[0][0]: tensor(-1.0454, grad_fn=<SelectBackward>)
pred[0][1]: tensor(1.1015, grad_fn=<SelectBackward>)
ans.data[0]: tensor(1)
ans.shape: torch.Size([1])
txt.shape: torch.Size([30, 1])
label.shape: torch.Size([1])
label.data[0]: tensor(1)

correct :  85
incorrect :  15
              precision    recall  f1-score   support

    negative     0.7966    0.9400    0.8624        50
    positive     0.9268    0.7600    0.8352        